<a href="https://colab.research.google.com/github/Zhangz5534/CS491/blob/main/POS_HMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download necessary resources

In [8]:
import numpy as np
import pandas as pd
import nltk

nltk.download('brown')
from nltk.corpus import brown


nltk.download('universal_tagset')

brown_tagged_sents = brown.tagged_sents(tagset='universal')


[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


In [9]:
brown_tagged_sents

[[('The', 'DET'), ('Fulton', 'NOUN'), ('County', 'NOUN'), ('Grand', 'ADJ'), ('Jury', 'NOUN'), ('said', 'VERB'), ('Friday', 'NOUN'), ('an', 'DET'), ('investigation', 'NOUN'), ('of', 'ADP'), ("Atlanta's", 'NOUN'), ('recent', 'ADJ'), ('primary', 'NOUN'), ('election', 'NOUN'), ('produced', 'VERB'), ('``', '.'), ('no', 'DET'), ('evidence', 'NOUN'), ("''", '.'), ('that', 'ADP'), ('any', 'DET'), ('irregularities', 'NOUN'), ('took', 'VERB'), ('place', 'NOUN'), ('.', '.')], [('The', 'DET'), ('jury', 'NOUN'), ('further', 'ADV'), ('said', 'VERB'), ('in', 'ADP'), ('term-end', 'NOUN'), ('presentments', 'NOUN'), ('that', 'ADP'), ('the', 'DET'), ('City', 'NOUN'), ('Executive', 'ADJ'), ('Committee', 'NOUN'), (',', '.'), ('which', 'DET'), ('had', 'VERB'), ('over-all', 'ADJ'), ('charge', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('election', 'NOUN'), (',', '.'), ('``', '.'), ('deserves', 'VERB'), ('the', 'DET'), ('praise', 'NOUN'), ('and', 'CONJ'), ('thanks', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('City

#Test

In [3]:
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
# Your example sentence
#sentence = "I go home by bus"
sentence = "The big cat sat on the mat in the kitchen"

In [5]:
# Tokenize the sentence and convert to lowercase
tokens = [word.lower() for word in word_tokenize(sentence)]

In [6]:
# Get the set of unique words in the Brown corpus
brown_words = set(brown.words())

In [7]:
# Check if all tokens are in the Brown corpus
all_in_corpus = all(token in brown_words for token in tokens)
print(all_in_corpus)

True


#Viterbi algorithm

In [10]:
brown_tagged_sents = brown.tagged_sents(tagset='universal')
brown_tagged_sents = [[('START', 'START')] + sentence + [('END', 'END')] for sentence in brown_tagged_sents]
# We adding to each sentence a start and end. We can think of 'Start' as a part of speech and also as a word.

brown_word_tags = [word_tag for sentence in brown_tagged_sents for word_tag in sentence]

# Extract the sequence of tags from the original list of (word, tag) pairs
tags = [tag for word, tag in brown_word_tags]

# Create pairs of consecutive tags
tag_pairs = [(tags[i], tags[i+1]) for i in range(len(tags)-1)]

# Create a CFD from the pairs of consecutive tags
cfd_tag_transitions = nltk.ConditionalFreqDist(tag_pairs)

# Convert the CFD into a CPD for transition probabilities
cpd_tag_transitions = nltk.ConditionalProbDist(cfd_tag_transitions, nltk.MLEProbDist)

tag_word_pairs = [(tag, word) for word, tag in brown_word_tags]
#reversing the ordering since we want to find prob(word|tag)

#Getting the conditional frequency distribution for the words which are tagged
cfd_word_given_tag=nltk.ConditionalFreqDist(tag_word_pairs)

# Create a ConditionalProbDist for emission probabilities
cpd_emission = nltk.ConditionalProbDist(cfd_word_given_tag, nltk.MLEProbDist)
# LaplaceProbDist is an alternative to MLE

In [11]:
import math

In [12]:
def viterbi(observed_words, cpd_tag_transitions, cpd_emission, states):
    # Initialize the dynamic programming table to store log probabilities
    V = [{}]
    path = {}

    # Initialize base case (t == 0)
    for state in states:
        emission_prob = cpd_emission[state].logprob(observed_words[0])
        if emission_prob == float('-inf'):
            emission_prob = -1e10  # Handling for unknown words
        V[0][state] = cpd_tag_transitions['START'].logprob(state) + emission_prob
        path[state] = [state]

    # Run Viterbi for t > 0
    for t in range(1, len(observed_words)):
        V.append({})
        newpath = {}

        for cur_state in states:
            # Check if state is 'START' or 'END'
            if cur_state in ['START', 'END']:
                continue

            # Select the state transition path with the maximum log probability
            (prob, state) = max(
                (V[t-1][prev_state] + cpd_tag_transitions[prev_state].logprob(cur_state) +
                 cpd_emission[cur_state].logprob(observed_words[t]), prev_state)
                for prev_state in states if prev_state not in ['START', 'END']
            )

            V[t][cur_state] = prob
            newpath[cur_state] = path[state] + [cur_state]

        # Don't need to remember the old paths
        path = newpath

    # Add a final step for transition to 'END' state
    logprob, state = max((V[len(observed_words) - 1][state] + cpd_tag_transitions[state].logprob('END'), state)
                         for state in states if state not in ['START', 'END'])
    return (logprob, path[state])

In [13]:
# Example sage 1
observed_words = ["I", "go", "home", "by","bus"]
states = ['NOUN', 'VERB', 'PRON', 'ADJ', 'ADV', 'ADP', 'CONJ', 'DET', 'NUM', 'PRT', 'X', 'START', 'END']
(prob, sequence) = viterbi(observed_words, cpd_tag_transitions, cpd_emission, states)
print(f"Probability of the best tag sequence: {prob}")
print(f"Best tag sequence: {sequence}")

Probability of the best tag sequence: -57.09103292304131
Best tag sequence: ['PRON', 'VERB', 'NOUN', 'ADP', 'NOUN']


In [14]:
# Example sage 2
observed_words = ["The", "big", "cat" ,"sat" ,"on", "the" ,"mat" ,"in", "the", "kitchen"]
states = ['NOUN', 'VERB', 'PRON', 'ADJ', 'ADV', 'ADP', 'CONJ', 'DET', 'NUM', 'PRT', 'X', 'START', 'END']
(prob, sequence) = viterbi(observed_words, cpd_tag_transitions, cpd_emission, states)
print(f"Probability of the best tag sequence: {prob}")
print(f"Best tag sequence: {sequence}")

Probability of the best tag sequence: -97.31195998201548
Best tag sequence: ['DET', 'ADJ', 'NOUN', 'VERB', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN']
